# MomaPy: a python library for molecular maps

The MomaPy library is a new python library for working with molecular maps such as SBGN maps.
Its key feature is its definition of a map, that is now formed of two entities: a model, that describes what concepts are represented, and a layout, that describes how these concepts are represented.
This definition is borrowed from SBML and its extensions layout/render, that allowed users to add a layout to an SBML model.
MomaPy aims at extending this definition to all types of molecular maps, and in particular to SBGN maps.

MomaPy offers the following features:
* support for SBGN PD and AF maps (read/write SBGN-ML with annotations, rendering information, and notes)
* decomposition of a map object into:
    - a model object;
    - a layout object;
    - a mapping between the model and layout objects' subelements
* map, model, layout and mapping objects comparison; fast object in set checking
* rendering of maps to images (SVG, PDF, JPEG, PNG, WebP) and other surfaces (e.g. GLFW window)
* support for styling and css like stylesheets (including effects such as shadows)
* automatic geometry and anchors (for arcs, shape borders)
* local positioning (e.g. right of shape, fit set of shapes)
* easy extension with new model and layout subelements

In [1]:
import momapy.io
import momapy.builder
import momapy.coloring
import momapy.styling
import momapy.utils

import momapy.sbgn.io.sbgnml
import momapy.sbgn.styling
import momapy.sbgn.utils

ModuleNotFoundError: No module named 'momapy'

In [ ]:
from momapy.demo.utils import display, display_at, show_room, macromolecule_toy, production_toy

# The `Map` object

In [ ]:
m = momapy.io.read("phospho1.sbgn")
display(m)

A `Map` object always contains a `Model`, a `Layout` and a `LayoutModelMapping` (that maps model elements to layout elements).
It may also have additional attributes depending on its nature. For example, an SBGN PD map also has an `id`, `notes`, and `annotations`.

In [ ]:
momapy.utils.pretty_print(m)

## The `Model` object

A `Model` may have an arbitrary number of attributes, depending on its nature.
For example, an `SBGNPDModel` has the following attributes: `entity_pools`, `processes`, `compartments`, `modulations`, `logical_operators`, `equivalence_operators`, `submaps` and `tags`, but also an `id`, `notes` and `annotations`.

In [ ]:
momapy.utils.pretty_print(m.model)

These attributes may be `ModelElement`s or collections of `ModelElements`. For example, the `entity_pools` attribute of an `SBGNPDModel` may contain zero or more `EntityPool`s.
We pick the first element from the collection:

In [ ]:
for e in m.model.entity_pools:
    break
momapy.utils.pretty_print(e)

Here, the element is a `Macromolecule`.
The data model for SBGN PD is built on a hierarchy of classes following the corresponding ontology.
Hence this element is also an `EntityPool`, and more generally a `ModelElement`:

In [ ]:
assert isinstance(e, momapy.sbgn.pd.EntityPool)
assert isinstance(e, momapy.core.ModelElement)

## The `Layout` object

A `Layout` is some sort of canvas that may contain other `LayoutElement`s that correspond to shapes that represent `ModelElement`s and that may be rendered.
In SBGN, `LayoutElement`s are either `Node`s, `Arc`s or `TextLayout`s.
The different `LayoutElement`s of a `Layout` are contained in its `layout_elements` attribute:

In [ ]:
momapy.utils.pretty_print(m.layout)

A `LayoutElement` always has a `drawing_elements` method that returns `DrawingElement`s that may be rendered using a `Renderer`.
The `DrawingElement`s are built on the fly based on the other attributes of the `LayoutElement`.
`DrawingElement`s are like SVG elements: they may represent paths, rectangle, ellipses, text..., and generally have the same attributes as their SVG counterpart (or a subset of them).
A `Layout` itself is a `Node`, whose returned unique `DrawingElement` ultimately represents a rectangle built from its `position`, `width` and `height` attributes.
The style of this rectangle depends on the styling attribute of the `Layout` (e.g. the `border_stroke`, `border_stroke_width`, and `border_fill` attributes):

In [ ]:
momapy.utils.pretty_print(m.layout.drawing_elements())

A `Layout` may contain other `LayoutElement`s, which themselves may contain other `LayoutElement`s, recursively forming a hierarchy of `LayoutElement`s, and thus of `DrawingElement`s.
While a `Layout` represents a `Model`, contained `LayoutElement`s represent `ModelElement`s contained by the `Model`.

We pick the first `LayoutElement` from the `SBGNPDLayout`:

In [ ]:
for l in m.layout.layout_elements:
    break
momapy.utils.pretty_print(l)

This element is a `MacromoleculeLayout`. Based on its `position`, `width` and `height` attributes, as well as on its styling attributes, it will produce a `DrawingElement` representing a rectangle with rounded corners, containing some text corresponding to its `label`. Since this element also contains other `LayoutElement`s in its `layout_elements` attribute, it will also produce the `DrawingElement`s of these contained `LayoutElement`s (here, a `StateVariableLayout`):

In [ ]:
display(l)

## The `LayoutModelMapping` object

A `LayoutModelMapping` is a mapping from `LayoutElement`s to `ModelElement`s and vice-versa.
It is used to map the `LayoutElement`s of a `Map` to the `ModelElement`s they represent.
It is intended to be as generic as possible so it maps sets of elements rather than elements themselves.

We pick the `ModelElement` mapped to the `MacromoleculeLayout` we had picked.
We know that there is only one `ModelElement` mapped to it so we can safely use the `unpack` option that removes the containing set:

In [ ]:
e = m.layout_model_mapping.get_mapping(l, unpack=True)[0]
momapy.utils.pretty_print(e)

# Equality and-sub relation for `Map`s, `Model`s, `Layout`s and `LayoutModelMapping`s

`Map`s, `Model`s, `Layout`s, `LayoutModelMapping`s, `ModelElement`s and `LayoutElement`s are developped so they can be easily compared.
Their identity relies on the value of a subset of their attributes (generally all attributes but their `id`, `notes` and `annotations`).
This way, two `Map`s can be easily compared.
Is is also possible to check whether a `Map` is a sub-map of another `Map` (not to be confused with SBGN PD's submap glyph).

## Equality

### Definition

Two `Map`s are equal if and only if:
* their `Model`s are equal;
* their `Layout`s are equal;
* and their `LayoutModelMapping`s are equal.

### Example

In [ ]:
m1 = momapy.io.read("phospho1.sbgn")
display(m1)

In [ ]:
m2 = momapy.io.read("phospho2.sbgn")
display(m2)

The two maps represent the exact same concepts, and thus have the same model. However, they do not have the same layout. Hence, the two maps are different.
This can be checked easily by comparing the `Map`, `Model`, `Layout` and `LayoutModelMapping` objects representing the two maps:

In [ ]:
assert m1 != m2
assert m1.model == m2.model
assert m1.layout != m2.layout
assert m1.layout_model_mapping != m2.layout_model_mapping

## Sub-map/model/layout/mapping

### Definition

A `Map` `M` is a sub-map of another `Map` `M'` if and only if:
* the `Model` of `M` is a sub-model of the `Model` of `M'`;
* the `Layout` of `M` is a sub-layout of the `Layout` of `M'`;
* and the `LayoutModelMapping` of `M` is a sub-mapping of the `LayoutModelMapping` of `M'`.


### Example 1

In [ ]:
m1 = momapy.io.read("phospho1.sbgn")
display(m1)

In [ ]:
m3 = momapy.io.read("phospho3.sbgn")
display(m3)

The second map is an excerpt of the first map:

In [ ]:
assert m3.is_submap(m1)
assert m3.model.is_submodel(m1.model)
assert m3.layout.is_sublayout(m1.layout)
assert m3.layout_model_mapping.is_submapping(m1.layout_model_mapping)

### Example 2

In [ ]:
m4 = momapy.io.read("phospho4.sbgn")
display(m4)

Because of the compartment, the model of the first map is not an excerpt of the model of the second map.
However, the layout of the first map is an excerpt of the layout of the second map:

In [ ]:
assert not m3.is_submap(m4)
assert not m3.model.is_submodel(m4.model)
assert m3.layout.is_sublayout(m4.layout)
assert not m3.layout_model_mapping.is_submapping(m4.layout_model_mapping)

# Frozen and builder objects

`Map`, `Model`, `Layout`, `ModelElement` and `LayoutElement` objects cannot be modified; they are frozen:

In [ ]:
m = momapy.io.read("phospho1.sbgn")
display(m)

In [ ]:
for l in m.layout.layout_elements:
    break
display(l)

In [ ]:
try:
    l.border_stroke_width = 3.0
except Exception as e:
    print(e)

This way they can be hashed, which is necessary to check whether a `Map` object belongs to a `set` efficiently, for example.
However, we want to be able to modify them programmatically (e.g., change the stroke width of the border of a shape).
To this end, each class has a corresponding builder class, that allows the production of objects that are not frozen.
Such objects may be built directly from the frozen objects:

In [ ]:
lb = momapy.builder.builder_from_object(l)
momapy.utils.pretty_print(lb)

In [ ]:
lb.stroke_width = 3.0
display(lb)

The frozen object may then be built back from the builder:

In [ ]:
l = momapy.builder.object_from_builder(lb)

In [ ]:
assert l.stroke_width == 3.0

The builder version of a map may be returned directly when reading the SBGN-ML file:

In [ ]:
mb = momapy.io.read("phospho1.sbgn", return_builder=True)
momapy.utils.pretty_print(mb)

# Reading and writing

`SBGNMap`s may be read from and written to SBGN-ML files using `read` and `write` functions:

In [ ]:
m = momapy.io.read("phospho1.sbgn")
momapy.io.write(m, "phospho1_output.sbgn", writer="sbgnml")

# Rendering

In [ ]:
m = momapy.io.read("phospho1.sbgn")
display(m)

`Map`s can be rendered in different formats using a simple render function:

In [ ]:
momapy.rendering.core.render_map(m, "phospho1.pdf", format_="pdf")
momapy.rendering.core.render_map(m, "phospho1.png", format_="png")
momapy.rendering.core.render_map(m, "phospho1.svg", format_="svg")
momapy.rendering.core.render_map(m, "phospho1.webp", format_="webp")
momapy.rendering.core.render_map(m, "phospho1.jpeg", format_="jpeg")

`Layout`s can be moved to the top left using the `top_left` option:

In [ ]:
momapy.rendering.core.render_map(m, "phospho1.pdf", format_="pdf", to_top_left=True)

In [ ]:
m1 = momapy.io.read("phospho1.sbgn")
m2 = momapy.io.read("phospho2.sbgn")
m3 = momapy.io.read("phospho3.sbgn")
m4 = momapy.io.read("phospho4.sbgn")

Multiple `Map`s can be rendered in one document using a simple function:

In [ ]:
momapy.rendering.core.render_maps([m1, m2, m3, m4], "phospho_multi.pdf", format_="pdf", multi_pages=True)

# Styling

## Styling `LayoutElement` objects

### Basic styling

Basic styling can be easily applied to `LayoutElement`s:

In [ ]:
mb = momapy.io.read("phospho1.sbgn", return_builder=True)
display(mb)

In [ ]:
for lb in mb.layout.layout_elements:
    break
display(lb)

In [ ]:
lb.border_fill = momapy.coloring.lightblue
lb.border_stroke = momapy.coloring.brown
lb.border_stroke_width = 3.0
lb.border_stroke_dasharray = (5, 5)
momapy.utils.pretty_print(lb.drawing_elements(), max_depth=3)

In [ ]:
display(lb)

The `label` of a `Node` can also be styled:

In [ ]:
lb.label.font_family = "Times"
lb.label.font_size = 30.0
lb.label.fill = momapy.coloring.red
lb.label.stroke = momapy.coloring.black
lb.label.stroke_width = 2.0
display(lb)

### Advanced styling

Advanced effects such as transformations (translation, rotation, ...) and filter effects can be applied to `LayoutElement`s:

In [ ]:
lb.transform = (momapy.geometry.Scaling(2, 1), momapy.geometry.Rotation(0.5, lb.position),)
lb.filter = momapy.drawing.Filter(effects=(momapy.drawing.DropShadowEffect(dx=3.0, dy=3.0, std_deviation=5.0, flood_opacity=0.5, flood_color=momapy.coloring.blue),))
display(lb)

## CSS-like style sheets

Styles may be applied to a `Map` using a `StyleSheet`. A `StyleSheet` can be built from a text document whose syntax is a subset of the CSS syntax.

In [ ]:
mb = momapy.io.read("phospho1.sbgn", return_builder=True)
display(mb)

There are pre-built `StyleSheet` objects for SBGN-ED and Newt-like styles, for colors, and for shadows:

In [ ]:
momapy.styling.apply_style_sheet(mb, momapy.sbgn.styling.newt)
display(mb)

Applying a `StyleSheet` to a `Map` may change the size of the nodes.
Some simple functions can be used to tidy the `Map`:

In [ ]:
momapy.sbgn.utils.newt_tidy(mb)
display(mb)

In [ ]:
momapy.styling.apply_style_sheet(mb, momapy.sbgn.styling.sbgned)
momapy.sbgn.utils.sbgned_tidy(mb)
display(mb)

In [ ]:
momapy.styling.apply_style_sheet(mb, momapy.sbgn.styling.cs_default)
display(mb)

In [ ]:
momapy.styling.apply_style_sheet(mb, momapy.sbgn.styling.fs_shadows)
display(mb)

These pre-built `StyleSheet`s are built from CSS-like text files:

In [ ]:
with open("../sbgn/styling/sbgned_no_cs.css") as f:
    for line in f.readlines()[:35]:
        print(line[:-1])

One may build a `StyleSheet` from a file and apply it to a `Map` with simple functions:

In [ ]:
with open("my_style_sheet.css", "w") as f:
    f.write("""
    SBGNPDLayout {
        border-stroke: red;
        border-fill: lightyellow;
    }
    
    MacromoleculeLayout {
        border-fill: green;
    }
    """)
my_style_sheet = momapy.styling.StyleSheet.from_file("my_style_sheet.css")
momapy.styling.apply_style_sheet(mb, my_style_sheet)
display(mb)

# Automatic geometry

`LayoutElement`s support automatic geometry: their "shape" can be automatically computed from the `DrawingElement`s they return, and be accessed with various methods.
These methods depend on the nature of the `LayoutElement` (`Node` or `Arc`).

## For `Node`s

In [ ]:
lb = macromolecule_toy()
display(lb)

### Anchors

`Node`s have anchor points, that are specific `Point`s on their border:

In [ ]:
lb.north_west()

In [ ]:
display_at(lb, lb.north_west())

All `Node`s have at least the following anchor points:
* `north_west`
* `north`
* `north_east`
* `east`
* `south_east`
* `south`
* `south_west`
* `west`
* `center`
* `label_center`

In [ ]:
show_room(momapy.sbgn.pd.MacromoleculeLayout)

In [ ]:
show_room(momapy.sbgn.pd.GenericProcessLayout)

In [ ]:
show_room(momapy.sbgn.pd.NucleicAcidFeatureMultimerLayout)

### Angles

`Node`s also have angle points:

In [ ]:
lb.self_angle(130)

In [ ]:
display_at(lb, lb.self_angle(130))

In [ ]:
show_room(momapy.sbgn.pd.MacromoleculeLayout, "self_angle")

In [ ]:
display_at(lb, lb.angle(130))

In [ ]:
show_room(momapy.sbgn.pd.MacromoleculeLayout, "angle")

## For `Arc`s

In [ ]:
lb = production_toy()
display(lb)

### Anchors

Analogously to `Node`s, `Arc`s have a few anchor points:

In [ ]:
lb.arrowhead_base()

In [ ]:
display_at(lb, lb.arrowhead_base())

All `Arc`s have at least the following anchor points:
* `end_point`
* `start_point`

`SingleHeadedArcs` have the following additional anchor points:
* `arrowhead_base`
* `arrowhead_tip`

And `DoubleHeadedArcs` have the following additional ones:
* `start_arrowhead_base`
* `start_arrowhead_tip`
* `end_arrowhead_base`
* `end_arrowhead_tip`

In [ ]:
show_room(momapy.sbgn.pd.ProductionLayout)

### Fraction

`Arc`s also have fraction points:

In [ ]:
lb.fraction(0.50)

In [ ]:
pos, angle = lb.fraction(0.50)
display_at(lb, pos)

In [ ]:
show_room(momapy.sbgn.pd.ProductionLayout, "fraction")

# Relative positioning

Automatic geometry enables positioning `LayoutElement`s relatively to one another:

In [ ]:
mb = momapy.io.read("phospho1.sbgn", return_builder=True)
display(mb)

In [ ]:
for lb in mb.layout.layout_elements:
    if hasattr(lb, "label") and lb.label is not None and lb.label.text == "B": # we select the layout for B
        eb = lb
    elif momapy.builder.isinstance_or_builder(lb, momapy.sbgn.pd.GenericProcessLayout): # we select the process layout
        pb = lb

In [ ]:
eb.position = momapy.positioning.above_left_of(pb, 200, 50) # eb's position is set 200 units above and 50 units left of pb's position
eb.label.position = eb.position
momapy.sbgn.utils.tidy(mb) # sets the arcs to the borders
display(mb)

The following functions are available:
* above_left_of
* above_of
* above_right_of
* right_of
* below_right_of
* below_of
* below_left_of
* left_of

For container `Node`s, the `fit` function is also available:

In [ ]:
momapy.positioning.fit(mb.layout.layout_elements, xsep=20, ysep=10)

Each of the above functions has a corresponding `set` function, which directly sets the returned value(s) to the correct `Node`'s attributes:

In [ ]:
mb.layout.border_stroke = momapy.coloring.red
momapy.positioning.set_fit(mb.layout, mb.layout.layout_elements, xsep=20, ysep=10)
display(mb)

The `set` functions have an `anchor` option that sets the relative target anchor of the `Node` receiving the new position (default is `center`):

In [ ]:
momapy.positioning.set_above_left_of(eb, pb, 200, 50, anchor="south_east") # eb's position is set such that its south_east anchor is 200 units above and 50 units left of pb's position
eb.label.position = eb.position
momapy.sbgn.utils.tidy(mb)
display(mb)

# Building new types of `Node`s and `Arc`s

New types of `Node`s and `Arc`s can be easily programmed.
Since the geometry is automatic, it is only required to program the general shape of the `LayoutElement` using `DrawingElement`s.

## New `Node`

In [ ]:
import dataclasses

In [ ]:
@dataclasses.dataclass(frozen=True)
class MyTriangle(momapy.core.Node):
    height: float = 30.0
    width: float = 30.0
    border_fill: momapy.coloring.Color = momapy.coloring.white
    border_stroke: momapy.coloring.Color = momapy.coloring.black
    
    def border_drawing_elements(self):
        actions = [
            momapy.drawing.MoveTo(self.position - (0, self.height / 2)), # top
            momapy.drawing.LineTo(self.position + (self.width / 2, self.height / 2)), # bottom right
            momapy.drawing.LineTo(self.position + (-self.width / 2, self.height / 2)), # bottom left
            momapy.drawing.ClosePath()
        ]
        path = momapy.drawing.Path(actions=actions)
        return [path]

In [ ]:
show_room(MyTriangle)

In [ ]:
show_room(MyTriangle, "angle")

## New `Arc`

In [ ]:
@dataclasses.dataclass(frozen=True)
class MyRectangleArrow(momapy.core.SingleHeadedArc):
    arrowhead_width: float = 10.0
    arrowhead_height: float = 10.0
    arrowhead_fill: momapy.coloring.Color | momapy.drawing.NoneValueType = momapy.coloring.white
    arrowhead_stroke: momapy.coloring.Color = momapy.coloring.black
    path_fill: momapy.coloring.Color | momapy.drawing.NoneValueType = momapy.drawing.NoneValue
    path_stroke: momapy.coloring.Color = momapy.coloring.black


    def arrowhead_drawing_elements(self):
        actions = [
            momapy.drawing.MoveTo(momapy.geometry.Point(0,0)), # we draw the arrowhead as if its base is at (0, 0)
            momapy.drawing.LineTo(momapy.geometry.Point(0, -self.arrowhead_height / 2)), # top left
            momapy.drawing.LineTo(momapy.geometry.Point(self.arrowhead_width, -self.arrowhead_height / 2)), # top right
            momapy.drawing.LineTo(momapy.geometry.Point(self.arrowhead_width, self.arrowhead_height / 2)), # bottom right
            momapy.drawing.LineTo(momapy.geometry.Point(0, self.arrowhead_height / 2)), # bottom left
            momapy.drawing.ClosePath()
        ]
        path = momapy.drawing.Path(actions=actions)
        return [path]

In [ ]:
show_room(MyRectangleArrow)

In [ ]:
show_room(MyRectangleArrow, "fraction")

New `LayoutElement`s can be even more easily programmed using the `Shape`s already available in the `meta` module:

In [ ]:
import momapy.meta.shapes

In [ ]:
@dataclasses.dataclass(frozen=True)
class MyDoubleRectangleArrow(momapy.core.DoubleHeadedArc):
    start_arrowhead_width: float = 10.0
    start_arrowhead_height: float = 10.0
    start_arrowhead_fill: momapy.coloring.Color | momapy.drawing.NoneValueType = momapy.coloring.white
    start_arrowhead_stroke: momapy.coloring.Color = momapy.coloring.black
    end_arrowhead_width: float = 20.0
    end_arrowhead_height: float = 20.0
    end_arrowhead_fill: momapy.coloring.Color | momapy.drawing.NoneValueType = momapy.coloring.white
    end_arrowhead_stroke: momapy.coloring.Color = momapy.coloring.black
    path_fill: momapy.coloring.Color | momapy.drawing.NoneValueType = momapy.drawing.NoneValue
    path_stroke: momapy.coloring.Color = momapy.coloring.black
    
    def start_arrowhead_drawing_elements(self):
        return momapy.meta.shapes.Rectangle(
            position=momapy.geometry.Point(self.start_arrowhead_width/2, 0),
            width=self.start_arrowhead_width,
            height=self.start_arrowhead_height
        ).drawing_elements()

    def end_arrowhead_drawing_elements(self):
        return momapy.meta.shapes.Rectangle(
            position=momapy.geometry.Point(self.end_arrowhead_width/2, 0),
            width=self.end_arrowhead_width,
            height=self.end_arrowhead_height
        ).drawing_elements()


In [ ]:
show_room(MyDoubleRectangleArrow)

# Ongoing and future work

### Ongoing work:
* Support for CellDesigner
* Support for SBML models (map skeleton) and layout/render
* Improve performance of geometry (currently slow)

### Future work:
* Support for background images and gradients
* Support for SBGN ER maps
* Support for BioPAX models
* Automatic XML/JSON format
* Automatic Neo4j storing for all types of maps (StonPy2)
* Developpment of converters